In [5]:
!pip install pycocotools

  Using cached pycocotools-2.0.8-cp312-cp312-macosx_10_9_universal2.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [27]:
import os
import numpy as np
from pycocotools.coco import COCO
from PIL import Image

def preprocess_image(image_path, target_size=(224, 224)):
    #Load an image, resize it to the target size, and normalize pixel values.
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image_array = np.array(image) / 255.0  # Normalize to [0,1]
    return image_array
    
def load_coco_annotations(coco_json_path, images_dir, target_size=(224, 224), max_images = 100):
    #From the COCO Json_File, load and preprocess the first 100 images/annotations
    coco = COCO(coco_json_path)
    images = []
    annotations = []
    
    #Getting the first 100 images from the files
    img_ids = coco.getImgIds()[:max_images]
    
    # Loop over the 100 images
    for img_id in img_ids:
        img_info = coco.loadImgs(img_id)[0] #Loads the first entry of the list of dictionaries containing metadata; Img_info has file name, width, height, etc.
        image_path = os.path.join(images_dir, img_info['file_name']) #Combines the image directory [With all images] with file name info retrieved in img_info
        
        # Preprocess the image
        processed_image = preprocess_image(image_path, target_size) #Refers to previously defined function
        images.append(processed_image) #Append the newly processed images to the images list.
        
        # Get annotations for the current image
        ann_ids = coco.getAnnIds(imgIds=img_id) #Retrieves annotation data associated with current image in img_id loop; stores it in ann_ids list
        img_annotations = [] #Creates list to store processed annotations for current image in img_id loop
        
        for ann in coco.loadAnns(ann_ids): #looping through the annotation data for each image.
            if 'bbox' in ann: #To preprocess the bound box annotation if it's present
                x, y, w, h = ann['bbox'] #the annotation bbox has x, y, width, height in the boundbox
                x_scale = target_size[0] / img_info['width'] 
                y_scale = target_size[1] / img_info['height']
                #BOTH of the above lines scales the bounding box coordinates to target size for x and y coordinates; creates two variables to apply to the scaling
                
                # Scale bounding boxes to the new image size
                scaled_bbox = [x * x_scale, y * y_scale, w * x_scale, h * y_scale] #Application of the scaling to target size.
                img_annotations.append(scaled_bbox) #Appends the new bound box coordinates to processed annotations for current image
        
        annotations.append(img_annotations) #annotations now contains the processed bbox coordinates
    
    return np.array(images), annotations #Return the preprocessed images and annotations
    
coco_json_path = '/Users/shreejith/suryaprojects/Dental_ML_Diagnosis_Project/Panoramic_Dental_Xray_Dataset/COCO/COCO/annotations/train_coco.json'
images_dir = '/Users/shreejith/suryaprojects/Dental_ML_Diagnosis_Project/Panoramic_Dental_Xray_Dataset/COCO/COCO/train'
target_size = (224, 224)
max_images = 100

images, annotations = load_coco_annotations(coco_json_path, images_dir, target_size, max_images=max_images)

print(f"Processed {len(images)} images and {len(annotations)} sets of annotations.")

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
Processed 100 images and 100 sets of annotations.
